In [ ]:
import jsonlines
import numpy as np

def load_results_jsonl(path):
    with jsonlines.open(path) as reader:
        remove_accuracy = []
        base_accuracy = []
        enhance_accuracy =  []
        remove_pro = []
        enhance_pro = []
        _data = [obj for obj in reader]
        for _ in _data:
            for one_data in _:
                remove_accuracy.append(one_data['remove_label'])
                base_accuracy.append(one_data['base_label'])
                enhance_accuracy.append(one_data['enhance_label'])
        remove_accuracy = np.array(remove_accuracy).squeeze(1)
        base_accuracy = np.array(base_accuracy).squeeze(1)
        enhance_accuracy = np.array(enhance_accuracy).squeeze(1)

        # remove_pro = remove_accuracy/base_accuracy
        # enhance_pro = enhance_accuracy/base_accuracy

    return remove_accuracy, base_accuracy, enhance_accuracy

    

# MAE-B_16

In [ ]:
import os
import numpy as np
JA_PATH = '/edit-MAE-B-ja'
INFLUENCE_PATTERN_PATH = '/edit-MAE-B-ip'
BASE_PATH = '/edit-MAE-B-base'
INTEGRATED_GRADIENTS_PATH = BASE_PATH

ip_remove_lst = []
ip_enhance_lst = []
ja_remove_lst = []
ja_enhance_lst = []
base_remove_lst = []
base_enhance_lst = []
base_base_lst = []

for class_idx in range(1000):
    influence_pattern_path = os.path.join(INFLUENCE_PATTERN_PATH, f"imgnet_all-{int(class_idx)}.rlt.jsonl")
    ja_path = os.path.join(JA_PATH, f"imgnet_all-{int(class_idx)}.rlt.jsonl")
    base_path = os.path.join(BASE_PATH, f"imgnet_all-{int(class_idx)}.rlt.jsonl")

    ip_remove, ip_base, ip_enhance = load_results_jsonl(influence_pattern_path)
    ja_remove, ja_base, ja_enhance = load_results_jsonl(ja_path)
    base_remove, base_base, base_enhance = load_results_jsonl(base_path)

    assert not (ip_base-ja_base).any()

    ip_remove_lst.append((ip_remove==class_idx).sum())
    ip_enhance_lst.append((ip_enhance==class_idx).sum())
    ja_remove_lst.append((ja_remove==class_idx).sum())
    ja_enhance_lst.append((ja_enhance==class_idx).sum())
    base_remove_lst.append((base_remove==class_idx).sum())
    base_enhance_lst.append((base_enhance==class_idx).sum())
    base_base_lst.append((base_base==class_idx).sum())

In [ ]:
print(f"remove influence pattern: {np.array(ip_remove_lst).mean()/50.0 - np.array(base_base_lst).mean()/50.0}")
print(f"remove joint attribution: {np.array(ja_remove_lst).mean()/50.0 - np.array(base_base_lst).mean()/50.0}")
print(f"remove base: {np.array(base_remove_lst).mean()/50.0 - np.array(base_base_lst).mean()/50.0}")

In [ ]:
print(f"base: {np.array(base_base_lst).mean()/50.0}")

In [ ]:
print(f"enhance influence pattern: {np.array(ip_enhance_lst).mean()/50.0 - np.array(base_base_lst).mean()/50.0}")
print(f"enhance joint attribution: {np.array(ja_enhance_lst).mean()/50.0 - np.array(base_base_lst).mean()/50.0}")
print(f"enhance base: {np.array(base_enhance_lst).mean()/50.0 - np.array(base_base_lst).mean()/50.0}")